# Выражения и условия в SQL. CRUD-операции

### Основные элементы языка SQL

СУБД управления базами данных PostgreSQL обрабатывает команды на языке SQL. Каждая команда складывается из элементов, как предложение из слов. Давайте разберём эти элементы, чтобы понимать, как СУБД читает и выполняет наши инструкции.

**Язык SQL включает семь ключевых элементов:**

1. **Ключевые слова** — это слова, которые зарезервировала система, чтобы использовать их как команды или их части. Они определяют действие, которое нужно выполнить. Например, ключевое слово `SELECT` говорить системе извлечить данные, `FROM` — указывает на источник, а `WHERE` — вводит условие отбора.

    Давайте посмотрим, как СУБД выполняет запрос c тремя ключевыми словами `SELECT`, `FROM` и `WHERE`:

In [ ]:
%%sql
SELECT make, year FROM car WHERE year > 2015 LIMIT 3;

Как мы видим, система взяла таблицу `car`, проверила условие `year > 2015` для каждой строки и вывела марку и год только для тех автомобилей, которые прошли фильтр. Этот пример показывает, как ключевые слова влияют на логику выборки значений.

2. **Идентификаторы** — это имена, которые мы даём объектам в базе: таблицам, столбцам и представлениям, чтобы СУБД могла искать по ним данные.
    
    Давайте проверим, какие столбцы есть в таблице `car`:

In [ ]:
%%sql
SELECT column_name, data_type FROM information_schema.columns
WHERE table_name = 'car' ORDER BY ordinal_position;

Система вывела точную структуру таблицу с шестью столбцами:
- `cid` — целое число;
- `make` — текст;
- `year` — целое число;
- `mileage` — целое число;
- `clsid` — текст;
- `res_number` — целое число. 

Теперь мы знаем, какие идентификаторы мы можем использовать. Давайте попробуем написать запрос, чтобы узнать данные о пробеге:

In [ ]:
%%sql
SELECT cid, make, mileage FROM car WHERE mileage < 50000 LIMIT 3;

3. **Выражения** — это комбинации значений, имен столбцов, функций и операторов, которые вычисляют, чтобы получить одно конкретное значение или набор данных.

    Давайте напишем выражение, которое переводит пробег из миль в километры:

In [ ]:
%%sql
SELECT make, mileage, mileage * 1.60934 AS mileage_km FROM car LIMIT 3;

Как мы видим, система взяла значение `mileage` для каждого автомобиля, умножила его на `1.60934` и вывела результат в новом столбце `mileage_km`. Это и другие выражения позволяют динамически обрабатывать данные, но не изменять саму таблицу.

4. **Условия** — это выражение, которое система проверяет как истину или ложь. Условия используют, чтобы фильтровать строки. Условия используют операторы сравнения `>`, `<`, `=`, а также логические связки `AND` и `OR`.

    Давайте попробуем написать условие с оператором `AND`, чтобы отобрать строки с автомобилями, которые выпустили между `2010` и `2020` годом, и чем пробег больше `100000`:

In [ ]:
%%sql
SELECT make, year, mileage FROM car
WHERE year BETWEEN 2010 AND 2020 AND mileage > 100000
LIMIT 3;

Система проверила каждую строку таблицы `car` и не включила ни одной строки в результат, потому что ни один автомобиль не соответствовал условию, которое мы задали.

5. **Переменные** — это объекты-контейнеры, которые используют, что временно хранить, изменять и передавать данные в скриптах, процедурах и функциях. Например, числа, строки и даты. 

6. **Комментарии** — это текст, который пишут разработчики, чтобы пояснить логику кода. Когда СУБД выполняет код, она игнорирует комментарии.

    Давайте посмотрим, как выглядят комментарии в SQL-коде:

In [ ]:
%%sql
-- Это однострочный комментарий. Его используют, чтобы прокомментировать одну строку.
SELECT * FROM car LIMIT 1;

/*
Это многострочный комментарий.
Его используют, чтобы прокомментировать блоки кода.
*/

Система выполнила только команду `SELECT * FROM car LIMIT 1` и вывела только одну полную строку из таблицы.

7. **NULL** — это специальное значение, которое означает отсутствие значения. Если вы читали первый конспект, то вы знаете, что это не ноль и не пустая строка. `NULL` показывает, что о данных нет информации, или что их нельзя применить. Напомню, чтобы проверить данные на значение `NULL`, используют специальные операторы `IS NULL` и `IS NOT NULL`.

Давайте напишем запрос и проверим, есть ли автомобили без номера резервации:

In [ ]:
%%sql
SELECT cid, make, res_number FROM car WHERE res_number IS NULL LIMIT 3;

Как мы видим, система не нашла автомобилей, у которых в столбце `res_number` есть `NULL`. Это означает, что нет никаких данных о том, есть ли бронь на автомобилях.

### Идентификаторы в SQL (1/3)

Мы разобрали основные элементы SQL и увидели, как СУБД работает с ключевыми словами и значениями. Теперь сосредоточимся на **идентификаторах, или именах объектов**. Мы уже использовали простые имена. Например, `car` и `mileage`. Но в реальной записи правила их записи сложнее. Давайте разберёмся, как система разрешает имена и как мы можем на них ссылаться. 

**Идентификатор** — это имя, которое мы даём объекту базы данных. Например, таблице, столбцу или представлению. СУБД использует идентификаторы, чтобы находить нужные данные среди таблиц и столбцов.

SQL позволяет записывать идентификаторы в разных форматах, а СУБД интепретирует их по строгим правилам.

**Давайте изучим четыре формата записи идентификаторов:**

1. **Простые идентификаторы** — это стандартные имена объектов, которые СУБД считает независимыми от регистра и приводит к нижнему регистру. Например, мы можем писать как `make`, так и `MAKE` или `Make`. СУБД распознает их как `make`.

In [ ]:
%%sql
SELECT make, MAKE, Make FROM car;

Как мы видим, СУБД создала три отдельные колонки и пронумеровала их. Значение в каждой из колонок одинаковое, потому что мы использовали простые идентификаторы. 

2. **Квалифицированные идентификаторы** — это полные имена, которые указывают путь к объекту через точку в формате `таблица.столбец`. Они устраняют неоднозначность, когда  в разных таблицах запроса встречаются одинаковые имена атрибутов.

    Давайте создадим простую вторую таблицу и испольуем полные имена, чтобы различить столбцы `cid`:

In [ ]:
%%sql
CREATE TEMP TABLE colors (cid INT, color TEXT);
INSERT INTO colors VALUES (1, 'Red'), (2, 'Blue');

SELECT car.cid, car.make, colors.color
FROM car
JOIN colors ON car.cid = colors.cid
WHERE car.cid <= 2;

В запросе столбец `cid` есть и в `car`, и в `colors`. Мы используем полные имена `car.cid` и `colors.cid` в условии `ON`. В `SELECT` мы явно указываем `car.cid`, чтобы система вывела идентификатор из таблицы автомобилей. Это показывает, как полные имена помогают избежать ошибок в данных.

3. **Разграниченные идентификаторы** — это имена, которые мы заключаем в двойные кавычки. СУБД сохраняет их регистр. Без кавычек СУБД не найдёт объект с именем в верхнем регистре. 

    Давайте напишем запрос и сравним закавыченные и незакавыченные идентификаторы:

In [ ]:
%%sql
SELECT cid, "cid", "CID" FROM car LIMIT 1;

СУБД нашла столбец `cid` для первых двух запросов, но для `"CID"` она выдала ошибку, потому что в таблице `car` нет столбца с именем в верхнем регистре. Кавычки устанавливают правила учета регистра для системы

4. **Идентификатор `*`, или звезда,** — это специальный символ `*`, который заменяет собой полный список всех атрибутов отношения. Он работает как сокращённая запись, которая позволяет выбрать все доступные столбцы. Например, `*` означает все столбцы всех таблиц в запросе, а `car.*` — все столбцы таблицы `car`.

    Давайте напишем запрос, в котором используем `*` и `car.*` 

In [ ]:
%%sql
SELECT * FROM car LIMIT 2;

Этот запрос возвращает все шесть столбцов таблицы `car`: `cid`, `make`, `year`, `mileage`, `clsid` и `res_number`. Звезда автоматически подставляет их вместо того, чтобы явно перечислять.

Теперь давайте создадим вторую таблицу, чтобы показать, как работает `car.*` в запросе, где соединим две таблицы. Итоговая таблица будет содержать информацию о классе автомобиля:

In [ ]:
%%sql
-- Создадим временную таблицу `class_info` для демонстрации
CREATE TEMP TABLE class_info AS
SELECT DISTINCT clsid, 
       CASE clsid 
           WHEN 'eco' THEN 'Economy' 
           WHEN 'lux' THEN 'Luxury' 
           ELSE 'Standard' 
       END AS description
FROM car 
WHERE clsid IS NOT NULL;

-- Теперь используем `car.*` для выбора всех столбцов из `car` класса и добавим описание класса
SELECT car.*, class_info.description
FROM car
JOIN class_info ON car.clsid = class_info.clsid
WHERE car.clsid IS NOT NULL
LIMIT 3;

В этом запросе `car.*` разворачивается во все столбцы основной таблицы. К ним добавляется только один столбец `description` из таблицы `class_info`, которую мы присоединили. Это позволяет быстро получить все данные из основной таблицы с дополнительной информацией из связанной таблицы. Кроме того, мы не перечисляли каждый столбец таблицы `car`.

Важно отметить, что идентификатор `*` может вернуть много лишних данных и снизить производительность базы данных, если в таблице много столбцов. Поэтому её нужно использовать аккуратно.

### Идентификаторы в SQL (2/3) + (3/3)

Мы разобрались, как СУБД различается идентифкаторы таблиц и столбцов. Но в болших базах данных таблицы организуют в схемы, или логические пространства, которые помогают структурировать объекты. Сейчас мы узнаем, как указывать схемы явно, и как они помогают искать таблицы.

Вспомним, что **схемы — это формальное описание структуры базы данных.** Теперь мы можем дополнить это определение и сказать, что **схемы — ещё и именованный контейнер, который хранит объекты базы данных.** Например, таблицы, представления и функции. Помимо того, что схема хранит эти объекты, она упорядочивает их и позволяет управлять доступом к этим объектам.

Например, в PostgreSQL схема по умолчанию имеет имя `public`. Когда мы пишем запрос `SELECT * FROM car`, СУБД ищет таблицу `car` сначала в схеме `public`, а затем по пути поиска `search_path`.

Если нам нужно обратиться к таблице из конкретной схемы, мы указываем её имя перед именем таблицы, но разделяем точкой: `имя_схемы.имя_таблицы`. Это правило работает как для представлений, так и для ссылок на столбцы.

Давайте проверим, в какой схеме находится наша таблица `car`, и попробуем явно указать схему в запросе:

In [ ]:
%%sql
SELECT table_schema, table_name
FROM information_schema.tables
WHERE table_name = 'car';

Мы видим, что схема, которая хранит нашу таблицу `car`, называется `public`. Теперь напишем запрос и укажем схему явно:

In [ ]:
%%sql
SELECT public.car.cid, public.car.make
FROM public.car
WHERE public.car.year > 2015
LIMIT 2;

Мы увидели, что СУБД корректно обработала полное имя `public.car`. Она нашла таблицу и вернула нам данные. Несмотря на то, что СУБД нашла бы таблицу и без `public.`, такой синтаксис становится обязательным, если в базе есть несколько таблиц `car` в разных схемах, иля путь поиска таблицы не включачет схему по умолчанию. 

СУБД позволяет определить, где искать таблицы и с помощью параметра `search_path`. Давайте посмотрим текущий путь поиска для нашего подключения:

In [ ]:
%%sql
SHOW search_path;

СУБД вернула нам список схем, которые обычно проверяют по порядку, когда встречает имя таблицы без схемы. Первым значением идёт `$user`, то есть схема с именем текущего пользователя, а затем `public`. Теперь мы лучше понимаем, почему наш запрос `SELECT * FROM car` работает — СУБД находит таблицу в схеме `public`.

Схемы позволяют нескольким пользователям или приложениям работать в базе данных и не мешать друг другу. Например, бухгалтерия может хранить таблицы в схеме `accounting`, а отдел продаж — в схеме `sales`. Когда запрос явно указывает `accounting.invoices`, СУБД точно знает, какую таблицу использовать.  

Давайте создадим временную схему и посмотрим, как система различает одноимённые таблицы в разных схемах:

In [ ]:
%%sql
CREATE SCHEMA IF NOT EXISTS demo;
CREATE TABLE demo.car AS SELECT * FROM car LIMIT 1;

SELECT 'public' AS schema_name, cid, make FROM public.car WHERE cid = 1
UNION ALL
SELECT 'demo' AS schema_name, cid, make FROM demo.car;

Мы создали новую схему `demo`, которая копирует одну строку из `public.car` в таблицу `demo.car`. После этого запрос обращается к двум разным схемам и показывает, что система может содержать разные данные в одноимённых таблицах под разными схемами. Именно поэтому в сложных окружениях необходимо явно указывать схемы.

### Выражения в SQL (1/6)

Мы разобрали, как система находит объекты по именам и схемам. Теперь перейдём к выражениям. Мы уже знаем, что это такое, но дополню, что именно выражения добавляют логику и гибкость нашим запросам. Также мы знаем, что каждое выражение возвращает одно значение для каждой обрабатываемой строки и имеет тип данных. Например, `integer`, `text`, `boolean`.

**Изучим пять основных типов выражений:**

1. **Колонка таблицы** — это выражение, которое возвращает значение столбца для текущей строки. Например, мы можем указывать столбец по имени `mileage`, или с идентификатором таблицы `car.mileage`. 

    Давайте напишем запрос, где используем выражения столбцы:

In [ ]:
%%sql
SELECT cid, car.make, year FROM car LIMIT 3;

Как мы видим, СУБД обработала `cid`, `car.make` и `year` как выражения, которые ссылаются на столбцы таблицы `car`. Для каждой из трёх строк она подставила значения из этих столбцов. Так выполняется любой запрос на выборку данных.

2. **Константа** — это выражение с фиксированным значением. Константы бывают:
    - числовые — `100`;
    - с плавающей точкой — `100.45`;
    - строковые — `строка`;
    - даты — `2024-02-13`.

Давайте напишем запрос и добавим константные выражения к данным из столбцов:

In [ ]:
%%sql
SELECT make, 2024 AS current_year, 'автомобиль' AS type FROM car LIMIT 3;

В результате мы получили значения столбца `make` и два новых столбца-константы — `current_year` и `type`. В каждой строке `current_year` всегда равно `2024`, а `type` всегда содержит строку `автомобиль`. Это показывает, как константы дополняют данные информацией. 

3. **Формула с операторами** — это арифметическое выражение с операторами `+`, `-`, `*`, `/`. СУБД вычисляет эти формулы для каждой строки с помощью значений столбцов и констант.

    Давайте запрос с выражением-формулой, где представим пробег в тысячах миль:

In [ ]:
%%sql
SELECT make, mileage, mileage / 1000.0 AS mileage_thousands FROM car LIMIT 3;

Для каждого автомобиля СУБД взяла значение `mileage`, поделили его на `1000.0` и вывела результат в новом столбце `mileage_thousands`. Например, пробег `85000` превратился в 85.0. Так выражения с операторами помогают проводить расчёты прямо в запросе.

4. **Функция** — это операция, которая принимает аргументы в виде других выражений и возвращает результат. В SQL есть много функций, которые позволяют работать с числами, строками, числами и данными.

    Давайте напишем запрос со строковыми функциями `UPPER()` и `LOWER()`, а также с функцию текущей даты `NOW()`.

In [ ]:
%%sql
SELECT make,
    UPPER(make) AS make_upper,
    LOWER(make) AS make_lower,
    NOW() AS query_time
FROM car
LIMIT 3;

В результате СУБД применила функцию `UPPER()` к каждому значению столбца `make` и поместила новые значения в новый столбец `make_upper`. Кроме того, система применила к значениям столбца `make` функцию `LOWER()` и создала отдельный столбец `make_lower`, куда поместила значения с нижним регистром. Функция `NOW()` вернула одну константу с меткой времени, в которой запрос выполнился для всех строк.

Кроме того, мы можем встраивать запрос `SELECT` внутрь выражения, если этот подзапрос возвращает ровно одно значение, то есть одну строку с одним столбцом. Такие выражения называют скалярными подзапросами.

Давайте напишем запрос, где используем подзапрос как выражение. Мы добавим средний пробег по всем автомобилям к каждой строке.

In [ ]:
%%sql
SELECT make, mileage,
    (SELECT AVG(mileage) FROM car) AS avg_mileage
FROM car
LIMIT 3;

Подзапрос `(SELECT AVG(mileage) FROM car) вычисляется один раз и возаращает одно число — средний пробег по всей таблице `car`. Система подставляем это число как константу в каждую строку результата. В выводе мы видим личный пробег автомобиля и общее среднее значение в одном контексте.

Настоящая сила SQL проявляется, когда мы комбинируем разные типы выражения в одной инструкции. Давайте попробуем это сделать:

In [ ]:
%%sql
SELECT 
    make,
    mileage,
    (mileage * 1.60934) / 1000 AS mileage_km_thousands,
    'Проверено: ' || TO_CHAR(NOW(), 'YYYY-MM-DD') AS status
FROM car
WHERE year > 2010
LIMIT 3;

**Давайте разберёмся подробнее, что находится в этом запросе:**
- `mileage` — выражение-столбец;
- `(mileage * 1.60934) / 1000` — формула, которая переводит пробег в килоемтры и выражает результат в тысячах;
- `'Проверено: ' || TO_CHAR(NOW(), 'YYYY-MM_DD'` — комбинация строковой константы, оператора конкатенации `||` и функции `TO_CHAR()`, которая форматирует результат функции `NOW()` в строку.

### Выражения в SQL (2/6)

Мы разобрали основные типы выражений в SQL. Теперь углубимся в числовые выражения, которые составляют основу расчётов в базе данных. Числовые выражения комбинируют числа, столбцы числового типа и функции, что позволяет выполнять арифметические и математические операции.

SQL поддерживает стандартные арифметические операторы и математические функции. Такие выражения работают с типами данных `integer`, `numeric` — точное число, `real` и `double precision` — числа с плавающей точкой. Давайте проверим каждый оператор и функцию на практике.

Давайте напишем запрос, в котором протестируем базовые операторы на константах:

In [ ]:
%%sql
SELECT 
  1 + 2 AS addition,
  5 * 4 AS multiplication,
  4 % 3 AS modulus,
  1 / 10 AS integer_division,
  1 / 10.0 AS float_division;

**В результе система вычисляет все пять выражений и выводит следующие результаты:**
- `addition` = `3`;
- `multiplication` = `20`;
- `modulus` = `1`;
- `integer_division` = 0 — когда оба операнда целые, система выполняет целочисленное деление и отбрасывает дробную часть;
- `float_division` = 0.1 — если хотя бы одини операнд имеет десятичную часть, то результатом будет десятичная дробь.

**SQL также позволяет округлять значения с помощью следующих функций:**
- `FLOOR()` — округлять вниз;
- `CEIL()` — округлять вверх;
- `ROUND()` — округлять до конкретной точки.

Давайте протестируем функции округление на константе `3.45`:

In [ ]:
%%sql
SELECT
    3.45 AS original,
    FLOOR(3.45) AS floor_result,
    CEIL(3.45) AS ceil_result,
    ROUND(3.45, 1) AS round_1_decimal;

**В результате система применила все функции к числу `3.45` и вывела следующие результаты:**
- `FLOOR(3.45)` возвращает `3` — это наибольшее целое, которое не превышает аргумент;
- `CEIL(3.45)` возвращает `4` — это наименьшее целое, не меньшее аргумента;
- `ROUND(3.45, 1)` возвращает `3.5` — это округлённое число до одного знака после запятой.

Теперь давайте применим числовые выражение к данным о пробеге автомобиля из таблицы `car`. Для этого мы используем столбец `mileage`:

In [ ]:
%%sql
SELECT
    cid,
    mileage,
    mileage / 1000 AS mileage_thousands_int,
    mileage / 1000.0 AS mileage_thousands_float,
    ROUND(mileage / 1000.0, 1) AS mileage_rounded,
    (ROUND(mileage / 1000.0, 1) + 5) * 10 AS calculated_score
FROM car
WHERE mileage IS NOT NULL
LIMIT 4;

В результате для каждого автомобиля система вычислила:
- `mileage_thousands_int` — целое число тысяч миль пробега;
- `mileage_thousands_float` — число тысяч миль пробега с плавающей точкой;
- `mileage_round` — результат округления до одного знака после запятой;
- `calculated_score` — результат комплексного выражения: `/` -> `ROUND() -> `+5` -> `*10`.

Мы узнали, как строить сложные вычисления между значениями столбцов с помощью операторов, а также математических и арифметических функций.

Теперь углубимся в целочисленное деление. Когда оба операнда имеют целочисленный тип, система отбрасывает дробную часть результата. Давайте напишем запрос, где покажем разницу между целочисленным и обычным делением:

In [ ]:
%%sql
SELECT
    7 / 2 AS integer_division_7_by_2,
    7.0 / 2 AS float_division_7_by_2,
    7 / 2.0 AS float_division_2_by_7,
    mileage / 7 AS mileage_div_7_int,
    mileage / 7.0 AS mileage_div_7_float
FROM car
WHERE cid = 1;

В выводе кода мы видим, как работает правило SQL о целочисленном деление. Давайте изучим результаты пяти выражений для автомобиля, где `cid = 1`:
1. `7 / 2` возвращает `3`, потому что система выполняет целочисленные деление и отбрасывает дробную часть `0.5`, так как оба операнда целые.
2. `7.0 / 2` возвращает `3.5`, потому что система сохраняет десятичную часть, так первый операнд `7.0` имеет тип с плавающей точкой.
3. `7 / 2.0` также возвращает `3.5`, потому что второй операнд `2.0` является числом с плавающей точкой.
4. `mileage / 7` возвращает `12142`, так как система снова применяет целочисленное деление к пробегу автомобиля.
5. `mileage / 7.0` возвращает `12142.857142857143` — тот же результат, но уже в десятичной дроби, потому что делитель `7.0` имеет типа с плавающей точкой.

### Выражения в SQL (3/6)

Мы разобрали числовые выражения и увидели, как система выполняет арифметические операции. Теперь перейдём к **строковым выражениям**. Они позволяют нам работать с текстовыми данными: объединять и изменять регистр, извлекать части строк и заменить их фрагменты. Это позволяет форматировать вывод, очищать данные и проще искать информацию.

Строковые данные управляют текстовыми данными типа `text` или `varchar`, а в SQL есть операторы и функции, которые преобразуют строки. Мы будем работать с данными столбца `make`, чтобы узнать на практике, как работают строковые выражения. Некоторые примеры вы могли видеть в предыдущих ячейках с кодом, сейчас мы остановимся на них подробнее.

**Рассмотрим четыре основных строковых выражений:**

1. **Конкатенация** — это операция, которая соединяет две или более строк в одну. Чтобы выполнить конкатенацию, в SQL используют оператор `||`.

    Чтобы увидеть, как работает конкатенация, давайте соединим марку автомобиля с её текстовым описанием:

In [ ]:
%%sql
SELECT
    make,
    make || ' автомобиль' AS description,
    'Марка: ' || make || ', год: '|| year::text AS full_info
FROM car
LIMIT 3;

В выводе кода мы видим три таблицы: `make`, `description` и `full_info`. Сначала система берёт значение `make` для каждого автомобиля, а затем выполняет следующие действия:
- в первом выражении система присоединяет строку ` автомобиль`, в результате чего получается столбец `description` с названием марки автомобиля и надписью `автомобиль`;
- во втором выражении система комбинирует константы `Марка: `, значение `make`, константу `', год: '` и преобразованный в текст год: `year::text`.

2. **Смена регистра** — это операция, которая позволяет преобразовать все строки в верхний регистр с помощью функции `UPPER()` и в нижний — с помощью функции `LOWER()`.

    Давайте напишем запрос, который будет менять регистр названий марок автомобилей:

In [ ]:
%%sql
SELECT
    make,
    UPPER(make) AS make_upper,
    LOWER(make) AS make_lower
FROM car
WHERE make IS NOT NULL
LIMIT 3;

Система применяет функции к каждому значению `make`, в результате чего `UPPER('Toyota')` возвращает `'TOYOTA'`, а `LOWER('Toyota')` возвращает `toyota`.

3. **Извлечение подстроки** — это операция, которая позволяет извлекать части строк с указанием позиции и длины.
    
    Извлечение подстроки включает в себя три функции:
    - `SUBSTRING(строка, начало[, длина])` — извлекает подстроку с заданной позиции;
    - `LEFT(строка, длина)` — извлекает указанное количество символов с начала;
    - `RIGHT(строка,  длина)` — извлекает указанное количество символов с конца.
    
        
    Давайте напишем запрос, который будет извлекать части из строковых данных:

In [ ]:
%%sql
SELECT
    make,
    SUBSTRING(make FROM 1 FOR 3) as first_3_chars,
    LEFT(make, 2) AS left_2,
    RIGHT(make, 2) AS right_2
FROM car
WHERE make IS NOT NULL
LIMIT 3;

В результате для каждой марки система возвращает первые три буквы, а также первые и последние две буквы.

4. **Замена фрагментов строки** — это операция, которая позволяет найти части подстроки в исходной строке и заменить их на другую подстроку с помощью функции `REPLACE`.

    Давайте напишем запрос, который заменит часть текста в марках автомобилей:

In [ ]:
%%sql
SELECT
    make,
    REPLACE(make, 'o', '0') AS replaced_o
FROM car
WHERE make LIKE '%o%'
LIMIT 3;

В результате мы получили значение, где `0` заменил букву `o`. 

Как и с числовыми выражениями, мы можем комбинировать строковые операции, чтобы решать сложные задачи.

Давайте напишем запрос, где скомбинируем несколько строковых операций:

In [ ]:
%%sql
SELECT 
  make,
  UPPER(LEFT(make, 1)) || LOWER(SUBSTRING(make FROM 2)) AS capitalized,
  'ID-' || REPLACE(UPPER(make), ' ', '_') || '-' || year::text AS vehicle_code
FROM car 
WHERE make IS NOT NULL
LIMIT 3;

В выводе кода мы видим комплексное преобразование значений таблиц `capitalized` и `vehicle_code`:
1. **Для `capitalized`** система берёт первый символ `make`, переводит его в верхний регистр, затем берёт остаток строки со второго символа и переводит в нижний регистр, и наконец соединяет обе части.
2. **Для `vehicle_code`** система начинает со строки 'ID-', затем переводит значение `make` в верхний регистр и заменяет в нём пробелы на подчёркивания, после чего добавляет дефис и год выпуска автомобиля как текст.

### Выражения в SQL (4/6)

SQL позволяет преобразовывать значения из одного типа данных в другой. Это называется приведением типов. Приведение типов полезно, когда нужно сравнить значения разных типов или подготовить данные для вычислений. Например, чтобы соединить число с текстом, сначала нужно привести число к строке. Стандарт SQL определяет для этого функцию `CAST()`, а PostgreSQL дополнительно поддерживает синтаксис с двойным двоеточием `::`.

**Стандартный синтаксис** выглядит так: `CAST(выражение AS ТипДанных)`. Этот способ работает в любой СУБД, которая следует стандарту SQL.

**Синтаксис PostgreSQL** записывается так: `выражение::ТипДанных`. Двойное двоеточие делает то же самое, но записывается компактнее. Этот способ работает только в PostgreSQL.

Рассмотрим основные варианты приведения типов:

| Стандартный синтаксис              | PostgreSQL-синтаксис         | Результат                        |
|-----------------------------------|-----------------------------|----------------------------------|
| `CAST(col1 AS date)`              | `col1::date`                | дата                             |
| `CAST(col1 AS timestamp)`         | `col1::timestamp`           | дата и время                     |
| `CAST(col1 AS int)`               | `col1::int`                 | целое число                      |
| `CAST(col1 AS varchar(255))`      | `col1::varchar(255)`        | строка длиной до 255 символов    |
| `CAST(col1 AS float)`             | `col1::float`               | число с плавающей точкой         |

Давайте проверим оба способа на практики. Напишем запрос, который преобразовывает год выпуска автомобиля из целого числа в текст:

In [ ]:
%%sql
SELECT
    year,
    CAST(year AS varchar) AS year_text_standard,
    year::varchar AS year_text_postgres
FROM car
LIMIT 3;

Запрос вернул три столбца: исходный `year` как целое число и два текстовых представления того же значения. Мы два раза привели значение столбца `year` к тексту в виде строки с кодом.

Теперь давайте преобразуем строку в дату:

In [ ]:
%%sql
SELECT
    '2024-02-13'::date AS date_short,
    CAST('2024-02-13' AS date) AS date_standard;

Система распознает строку и преобразует её в тип `date`. Оба выражения возвращают одну и ту же дату.